# __Web Scraping desde la paractica: Paginas Dinamicas__

Las paginas web dinamicas renderizan el HTML conforme vamos haciendo scroll de la misma. Esto genera un gran desafio a la hora de scrapear ya que no tiene definido un limite. Para solucionarlo, vamos a usar Selenium.

## Importar librerias

In [20]:
import requests

## Obtener sitio web dinamico desde una peticion

In [21]:
# definir url
url = 'https://quotes.toscrape.com/scroll'

# realizar la peticion GET
website = requests.get(url)

# mostrar la web en texto
print(website.text)

<!DOCTYPE html>
<html lang="en">
<head>
	<meta charset="UTF-8">
	<title>Quotes to Scrape</title>
    <link rel="stylesheet" href="/static/bootstrap.min.css">
    <link rel="stylesheet" href="/static/main.css">
    
    
</head>
<body>
    <div class="container">
        <div class="row header-box">
            <div class="col-md-8">
                <h1>
                    <a href="/" style="text-decoration: none">Quotes to Scrape</a>
                </h1>
            </div>
            <div class="col-md-4">
                <p>
                
                    <a href="/login">Login</a>
                
                </p>
            </div>
        </div>
    
<div class="row">
    <div class="col-md-8">
        <div class="quotes"></div>
    </div>
</div>
<div id="loading" style="background-color: #eeeecc"><h5>Loading...</h5></div>
<script src="/static/jquery.js"></script>
<script>
    $(function(){
        var page = 1, tag = null, hasNextPage = true;
        function appendQu

> El codigo de la pagina presenta un codigo en javascript que permite que aparezcan los elementos a medida que se scrollea.

## Dependencias

In [22]:
import time
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.chrome.service import Service

## Configuracion del Servidor y Driver 

In [23]:
# ruta al chromedriver
chrome_driver_path = '/usr/local/bin/chromedriver-linux64/chromedriver'

# configurar opciones para visualizar el paso a paso de la interaccion con el navegador
options = webdriver.ChromeOptions()
options.add_argument('--start-maximized') # pantalla completa
options.add_argument('--disable-blink-features=AutomationControlled') # menos detectable

# Crear el servidor y el driver
service = Service(executable_path=chrome_driver_path)
driver = webdriver.Chrome(service=service, options=options)


## Definir url

In [24]:
# definir url
url = 'https://quotes.toscrape.com/scroll'

# realizar peticion GET
driver.get(url)

# definir un tiempo de espera para que se renderice el HTML
time.sleep(3)

## Manipular HTML renderizado

In [25]:
# obtener HTML renderizado
html = driver.page_source
driver.quit()

# procesar el HTML con beautifulSoup
soup = BeautifulSoup(html, 'html.parser')
quotes = soup.select('div.quote')

print('Citas encontradas: ', len(quotes))

for quote in quotes:
    text = quote.find('span', class_='text').get_text()
    autor = quote.find('small', class_='author').get_text()
    print(f'{text} - {autor}')

Citas encontradas:  10
“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.” - Albert Einstein
“It is our choices, Harry, that show what we truly are, far more than our abilities.” - J.K. Rowling
“There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.” - Albert Einstein
“The person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably stupid.” - Jane Austen
“Imperfection is beauty, madness is genius and it's better to be absolutely ridiculous than absolutely boring.” - Marilyn Monroe
“Try not to become a man of success. Rather become a man of value.” - Albert Einstein
“It is better to be hated for what you are than to be loved for what you are not.” - André Gide
“I have not failed. I've just found 10,000 ways that won't work.” - Thomas A. Edison
“A woman is like a tea bag; you never know how strong it is until it's in hot